In [4]:
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import glob
from scipy.misc import imread 
from scipy.misc import imresize
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Convolution2D, MaxPooling2D, Conv2D, MaxPool2D, Lambda
from keras.layers import BatchNormalization, LeakyReLU
from keras.utils import np_utils
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

In [47]:
# load data
file_names_dict = dict()
for i in [0, 1, 2]:
    image_files = glob.glob(f"/tmp/tl_training/{i}/*[!a-z].jpg")
    image_files = image_files[0:min(550, len(image_files))]
    file_names_dict[i] = image_files

   

In [48]:
data_dict = dict()
for key in file_names_dict:
    print(len(file_names_dict.get(key)))
    fnames = file_names_dict.get(key)
    images = [imresize(imread(x),(300, 400)) for x in fnames]
    data_dict[key] = images
    
    

550
550
550
550


In [49]:
X = []
Y = []
for key in data_dict:
    x = np.array(data_dict.get(key))
    y = np.ones(shape=x.shape[0]) * key
    X.append(x)
    Y.append(y)

In [50]:
X_train = np.vstack((X[0], X[1], X[2], X[3]))
Y_train = np.hstack((Y[0], Y[1], Y[2], Y[3]))

In [51]:
Y_train = np.hstack((Y[0], Y[1], Y[2], Y[3]))

In [52]:
print(X_train.shape)
print(Y_train.shape)

(2200, 300, 400, 3)
(2200,)


In [53]:
del X
#del Y
del data_dict
del file_names_dict

NameError: name 'red' is not defined

In [2]:
model = Sequential

model = Sequential([
    Lambda(lambda x: x / 255 - 0.5, input_shape=(300, 400, 3)),
    Conv2D(8, kernel_size=(5, 5), strides=(2,2)),
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    MaxPool2D(pool_size=(2,2), strides=(2,2)),
    Conv2D(16, kernel_size=(3, 3), strides=(1,1)),
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    MaxPool2D(pool_size=(2,2), strides=(2,2)),

    Conv2D(32, kernel_size=(3, 3), strides=(2, 2)),
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    Flatten(),
    Dense(55),
    Dense(4, activation='softmax')
])

model = Sequential([
    Lambda(lambda x: x / 255 - 0.5, input_shape=(300, 400, 3)),
    Conv2D(3, kernel_size=(5, 5), strides=(2,2)),
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    MaxPool2D(pool_size=(2,2), strides=(2,2)),
    Conv2D(9, kernel_size=(3, 3), strides=(2,2)),
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    MaxPool2D(pool_size=(2,2), strides=(2,2)),
    Dropout(0.25),

    Conv2D(18, kernel_size=(3, 3), strides=(1, 1)),
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    Flatten(),
    Dense(45),
    Dense(3, activation='softmax')
])


model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])
print(model.output_shape)

(None, 3)


In [5]:
# training
datagen = ImageDataGenerator(
    rotation_range = 30,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.1,
    zoom_range = 0.4,
    horizontal_flip=True,
    vertical_flip=True
)
test_datagen = ImageDataGenerator()

In [6]:
data_generator = datagen.flow_from_directory('/home/michael/tl_training',
          target_size=(300,400),
          batch_size=32,
          class_mode="categorical")

val_generator = test_datagen.flow_from_directory('/home/michael/tl_val',
        target_size=(300,400),
        batch_size=32,
        class_mode="categorical")

Found 360 images belonging to 3 classes.
Found 3264 images belonging to 3 classes.


In [ ]:
for i in range(100):
    model.fit_generator(data_generator, steps_per_epoch=210, epochs=10,
    validation_data=val_generator, validation_steps=300)

    model.save('../light_classification/models/whole_image_model_gpu{}.h5'.format(i))

Epoch 1/10
210/210 [==============================] - 177s - loss: 2.3665 - acc: 0.3695 - val_loss: 1.1524 - val_acc: 0.3334
Epoch 2/10
210/210 [==============================] - 174s - loss: 0.9596 - acc: 0.5537 - val_loss: 4.9118 - val_acc: 0.3343
Epoch 3/10
210/210 [==============================] - 173s - loss: 0.3446 - acc: 0.8662 - val_loss: 2.2566 - val_acc: 0.5578
Epoch 4/10
210/210 [==============================] - 173s - loss: 0.1728 - acc: 0.9414 - val_loss: 1.4255 - val_acc: 0.7455
Epoch 5/10
210/210 [==============================] - 173s - loss: 0.0984 - acc: 0.9649 - val_loss: 0.6595 - val_acc: 0.7995
Epoch 6/10
210/210 [==============================] - 173s - loss: 0.1196 - acc: 0.9630 - val_loss: 0.8235 - val_acc: 0.8182
Epoch 7/10
210/210 [==============================] - 173s - loss: 0.0596 - acc: 0.9807 - val_loss: 0.4631 - val_acc: 0.8693
Epoch 8/10
210/210 [==============================] - 172s - loss: 0.0458 - acc: 0.9853 - val_loss: 0.4336 - val_acc: 0.9065


output node names are:  ['output_node0', 'output_node1', 'output_node2', 'output_node3']


TensorShape([Dimension(None), Dimension(None)])

INFO:tensorflow:Froze 18 variables.
Converted 18 variables to const ops.


'./tl_dector.pb'